## Loading and cleaning

In [14]:
import sys
sys.path.insert(0, '../repository')

In [15]:
import database
import pandas as pd
import numpy as np
import glob
import re

pd.set_option('display.max_info_columns', 200)

In [16]:
conn = database.DatabaseConnection(db_name='transformation-index').connection

In [17]:
# Reading the iso codes.
f = open('../repository/DUMP/ISO CODE.csv', 'r')
iso_codes = pd.read_csv(f)
files = [f for f in glob.glob("../repository/RAW/BTI*")]
dfs = []
for i, f in enumerate(files):
    year = re.search("(\d{4})", f).group(0)
    with open(f, mode="r", encoding="utf-8"):
        _df = pd.read_csv(f, sep=";", index_col=False, decimal=',')
        regions = {
            k.strip(): v.strip()
            for k, v in [el.split("|") for el in _df.columns[0].split("\n")[1:]]
        }
        _df = _df.rename(str.strip, axis="columns")
        _df = _df.dropna(axis=1, how="all")
        _df["Region"] = _df["Region"].astype(str)
        _df["Region"] = _df["Region"].replace(regions)
        _df = _df.rename({_df.columns[0]: "Country"}, axis="columns")
        _df["Year"] = year
        _df = _df.replace(['-', '?'], np.nan)
        _df = _df.convert_dtypes()
        _df = _df[[*_df.columns[:2], "Year", *_df.columns[2:]]] # Reorder Year column
        _df = _df.loc[:,~_df.columns.duplicated()].copy() # type: ignore # Remove Year duplicates
        to_drop = []
        ptrn = re.compile('Trend.*')
        for i, col in enumerate(_df.columns):
            is_rm = re.match(ptrn, col)
            if is_rm:
                to_drop.append(i + 1) 
            try:
                _df[col] = _df[col].str.replace(',', '.').astype(float)
                try:
                    _df[col] = _df[col].astype(int)
                except ValueError:
                    pass
            except (ValueError, AttributeError):
                continue
        _df = _df.drop([_df.columns[i] for i in to_drop], axis=1) # Remove trending arrows
        dfs.append(_df)

In [18]:
ptrn = re.compile(r'^\w+.*\|\s') # Removing prefixes, such as 'Q1 | Stateness' or 'Q1.1 | Monopoly on the use of force'
rm_prefix = lambda x: ptrn.sub('', x).strip()
cols = dfs[1].rename(rm_prefix, axis='columns').columns
for df in dfs[1:]:
    df.columns = cols
df = pd.concat([*dfs[1:]])


In [19]:
same_col_name = lambda c: re.search(
    r"\s\w+\.\d", c
)  # Columns like 'Status Index.1', or 'Governance Index.1'
ptrn = re.compile(r"\.\d$")  # The '.1' at the end of the column name
for i, col in enumerate(df.columns):
    mtch = same_col_name(col)
    if mtch:
        df = df.rename(
            {
                df[col].name: ptrn.sub(" Score", df[col].name),
                df[df.columns[i + 1]].name: ptrn.sub(" Category", df[col].name),
                df[df.columns[i + 2]].name: ptrn.sub(" Status", df[col].name),
            },
            axis="columns",
        )

## Spliting

In [20]:
f = open('../repository/DUMP/ISO CODE.csv', 'r')
iso_codes = pd.read_csv(f)
iso_codes['id'] = pd.Series(range(len(iso_codes)), name='id')

In [21]:
df = df.merge(iso_codes.iloc[:, [-1, 0, 2]], left_on="Country", right_on="name")
df = df.drop(axis=1, columns=["Country", "name", "alpha-3", "Region"])
df = df.rename(columns={"id": "country_id"})
df = df[["country_id", "Year", *df.columns]]
df = df.loc[:, ~df.columns.duplicated()].copy()

In [22]:
gov_col = [
    "Free and fair elections",
    "Effective power to govern",
    "Separation of powers",
    "Independent judiciary",
    "Prosecution of office abuse",
    "Performance of democratic institutions",
    "Commitment to democratic institutions",
    "Level of Socioeconomic Development",
    "Socioeconomic barriers",
    "Monetary stability",
    "Fiscal stability",
    "Welfare Regime",
    "Social safety nets",
    "Equal opportunity",
    "Economic Performance",
    "Output strength",
    "Sustainability",
    "Environmental policy",
    "Education / R&D policy",
    "Level of Difficulty",
    "Structural constraints",
    "Conflict intensity",
    "GNI p.c. Atlas method rescaled",
    "UN Educ. Index rescaled",
    "BTI Stateness & Rule of Law",
    "Steering Capability",
    "Prioritization",
    "Implementation",
    "Policy learning",
    "Resource Efficiency",
    "Efficient use of assets",
    "Policy coordination",
    "Anti-corruption policy",
    "Consensus-Building",
    "Consensus on goals",
    "Anti-democratic actors",
    "Cleavage / conflict management",
    "Civil society participation",
    "Reconciliation",
    "International Cooperation",
    "Effective use of support",
    "Credibility",
    "Regional cooperation",
]

sg_col = df.columns.difference(gov_col, sort=False)

In [23]:
freedom = df.loc[:, ['Year', 'country_id', *gov_col]]
freedom = freedom
sg = df.loc[:, sg_col]

freedom = freedom.rename(str.lower, axis="columns")
freedom.columns = freedom.columns.str.replace(" ", "_")
sg = sg.rename(str.lower, axis="columns")
sg.columns = sg.columns.str.replace(" ", "_")

In [24]:
# freedom.to_csv('../repository/DUMP/Transformation Index/TI - Freedom.csv', sep=';', index=False, decimal='.')
# sg.to_csv('../repository/DUMP/Transformation Index/TI - State Governance.csv', sep=';', index=False, decimal='.')
freedom.to_sql('freedom', con=conn, if_exists='replace', index_label='id')
sg.to_sql('state_governance', con=conn, if_exists='replace', index_label='id')

360